In [54]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
customers= pd.read_csv('/content/Customers.csv')

In [56]:
products= pd.read_csv('/content/Products.csv')

In [57]:
transactions= pd.read_csv('/content/Transactions.csv')

In [58]:
data = transactions.merge(customers, on='CustomerID',how='left').merge(products, on='ProductID',how='left')

In [59]:
customer_transactions = data.groupby(['CustomerID', 'ProductID'])['Quantity'].sum().unstack(fill_value=0)

In [60]:
similarity_matrix = cosine_similarity(customer_transactions)

In [61]:
similarity_df = pd.DataFrame(similarity_matrix, index=customer_transactions.index, columns=customer_transactions.index)

In [62]:
#recommended top 3 similar customer
def recommend_similar_customers(input_customer_id, similarity_df):
    # Validate if the customer ID exists
    if input_customer_id not in similarity_df.index:
        return f"CustomerID {input_customer_id} not found in the dataset. Please check the input."

    # Compute top 3 similar customers excluding the customer itself
    similar_customers = similarity_df.loc[input_customer_id].sort_values(ascending=False)[1:4]  # Exclude self
    recommendations = [
        {"CustomerID": cust_id, "SimilarityScore": round(score, 3)}
        for cust_id, score in zip(similar_customers.index, similar_customers.values)
    ]
    return recommendations

In [63]:
recommendations = recommend_similar_customers(user_input, similarity_df)

In [64]:
#user_input= input("Enter CustomerID: ").strip()

CODE TO STORE DATA INTO CSV FILE

In [65]:
user_input= input("Enter CustomerID: ").strip()
print("\nTop 3 similar customers:")
if isinstance(recommendations, str):  # If the function returned an error message
    print(recommendations)
else:
    for idx, rec in enumerate(recommendations, start=1):
        print(f"{idx}. CustomerID: {rec['CustomerID']}, Similarity Score: {rec['SimilarityScore']}")

Enter CustomerID: C0025

Top 3 similar customers:
1. CustomerID: C0097, Similarity Score: 0.548
2. CustomerID: C0194, Similarity Score: 0.47
3. CustomerID: C0199, Similarity Score: 0.438


In [68]:


# Function to save recommendations to a CSV file
def save_recommendations_to_csv(input_customer_id, recommendations):
    recommendations_df = pd.DataFrame(recommendations)
    filename = f"Kritika_Jain_Lookalike_.csv"
    recommendations_df.to_csv(filename, index=False)
    print(f"\nRecommendations saved to {filename}")




In [69]:
# Output recommendations and save to CSV
if isinstance(recommendations, str):
    print(recommendations)
else:

    for idx, rec in enumerate(recommendations, start=1):
        print(f"{idx}. CustomerID: {rec['CustomerID']}, Similarity Score: {rec['SimilarityScore']}")


    save_recommendations_to_csv(user_input, recommendations)

1. CustomerID: C0097, Similarity Score: 0.548
2. CustomerID: C0194, Similarity Score: 0.47
3. CustomerID: C0199, Similarity Score: 0.438

Recommendations saved to Kritika_Jain_Lookalike_.csv


In [70]:
#similarity_df.head()